In [1]:
# Import WRDS library
import wrds
import pandas as pd

# Extracting data from Refinitiv Quantitative Analytics using WRDS

This is a tutorial and demonstration on how to extract stock prices using Refinitiv via WRDS.<br>
We are using Refinitiv in our project due to the convenience and wide data availability.<br>

In our project, we will extract the folloiwng data:
1. Market Price data for a single stock
2. Shares Outstanding for a single stock

Link to reference document: 
- Reference manual: https://wrds-www.wharton.upenn.edu/documents/1505/QADDatabaseSchemaDatastreamv1-1-6-20190530-.pdf
Requires logging into WRDS

In [2]:
# Establish live connection; requires user login (passwords will be masked)
db = wrds.Connection()

Enter your WRDS username [leeca]:cadenlee
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: n
You can create this file yourself at any time
with the create_pgpass_file() function.
Loading library list...
Done


## Here, we shall familiarize ourselves with the required tables.

The library name is `tr_ds_equities`.<br>
There are 4 key data tables that we will need to access, as follows:
1. `ds2isinchg`: The isin of the security. We need ISIN as our stock universe is identified with ISN (we can also use SEDOL as an alternative)
2. `ds2security`: This table contains the reference info of the security, including the name of the security and the infocode required. The infocode is the identifier we use to extract data from other tables.
3. `wrds_ds2dsf`: Datastream table for the daily stock prices. Queried with infocode
4. `ds2numshares`: Number of shares outstanding. Updated at every corporate action event date. Requried for market cap calculation.

In [3]:
db.describe_table(library = 'tr_ds_equities', table = 'ds2isinchg' )

Approximately 218661 rows in tr_ds_equities.ds2isinchg.


,name,nullable,type,comment
0,dsseccode,True,"NUMERIC(11, 0)",None
1,startdate,True,DATE,None
2,enddate,True,DATE,None
3,isin,True,VARCHAR(12),None
4,isin2,True,VARCHAR(12),None
5,licflagc,True,SMALLINT,None


In [4]:
db.describe_table(library = 'tr_ds_equities', table = 'ds2security' )

Approximately 192096 rows in tr_ds_equities.ds2security.


,name,nullable,type,comment
0,dsseccode,True,"NUMERIC(11, 0)",None
1,dssctycode,True,VARCHAR(13),None
2,dscmpycode,True,"NUMERIC(11, 0)",None
3,ismajorsec,True,VARCHAR(2),None
4,dssecname,True,VARCHAR(91),None
5,isocurrcode,True,VARCHAR(3),None
6,divunit,True,VARCHAR(7),None
7,primqtsedol,True,VARCHAR(7),None
8,primexchmnem,True,VARCHAR(3),None
9,primqtinfocode,True,"NUMERIC(11, 0)",None


In [5]:
db.describe_table(library = 'tr_ds_equities', table = 'wrds_ds2dsf' )

Approximately 524542848 rows in tr_ds_equities.wrds_ds2dsf.


,name,nullable,type,comment
0,infocode,True,"NUMERIC(11, 0)",None
1,dscode,True,VARCHAR(13),None
2,region,True,VARCHAR(7),None
3,regcodetypeid,True,INTEGER,None
4,currency,True,VARCHAR(3),None
5,exchintcode,True,INTEGER,None
6,isprimqt,True,SMALLINT,None
7,covergcode,True,VARCHAR(1),None
8,statuscode,True,VARCHAR(1),None
9,typecode,True,VARCHAR(5),None


In [6]:
db.describe_table(library = 'tr_ds_equities', table = 'ds2numshares' )

Approximately 11926813 rows in tr_ds_equities.ds2numshares.


,name,nullable,type,comment
0,infocode,True,"NUMERIC(11, 0)",None
1,eventdate,True,DATE,None
2,numshrs,True,DOUBLE_PRECISION,None
3,licflag,True,INTEGER,None


## Demo 1: Retrieving stock prices

In [3]:
ISIN = 'GB00BJP5HK17' # AAPL

In [4]:
# SQL script for price extraction
sql_script =\
'''
SELECT
    C.marketdate AS trade_date,
    A.dsseccode AS security_code,
    A.dssecname AS security_name,
    A.primexchmnem AS primary_exchange,
    A.primqtinfocode AS refinitiv_code,
    B.isin AS isin_code,
    C.currency AS currency,
    C.open AS open,
    C.high AS high,
    C.low AS low,
    C.close AS close,
    C.volume AS volume,
    C.statuscode AS status

    
FROM
    tr_ds_equities.ds2security A

INNER JOIN
    tr_ds_equities.ds2isinchg B
ON
    A.dsseccode = B.dsseccode

INNER JOIN
    tr_ds_equities.wrds_ds2dsf C
ON
    A.primqtinfocode = C.infocode
    
WHERE
    B.isin = '{}'
AND C.marketdate > TO_DATE('01/12/2022', 'DD/MM/YYYY')

ORDER BY C.marketdate
'''

In [5]:
# Extract security prices
db.raw_sql(sql_script.format(ISIN))

,trade_date,security_code,security_name,primary_exchange,refinitiv_code,isin_code,currency,open,high,low,close,volume,status
0,2022-12-02,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.5130,0.540000,0.499500,0.5290,7968660.0,S
1,2022-12-05,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.5300,0.541000,0.491000,0.5010,3904866.0,S
2,2022-12-06,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.5010,0.521000,0.490500,0.5100,3564001.0,S
3,2022-12-07,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.5220,0.522000,0.500000,0.5010,5302637.0,S
4,2022-12-08,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.5000,0.511000,0.466500,0.4700,5146215.0,S
5,2022-12-09,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.4700,0.478000,0.460000,0.4640,3121924.0,S
6,2022-12-12,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.4640,0.469000,0.380000,0.3840,17946422.0,S
7,2022-12-13,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.3840,0.410000,0.367500,0.3915,18205372.0,S
8,2022-12-14,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.3885,0.393500,0.373000,0.3790,8818984.0,S
9,2022-12-15,236079.0,HOME REIT,LON,330524.0,GB00BJP5HK17,GBP,0.3800,0.385360,0.370000,0.3850,8042074.0,S


## Demo 2: Retrieving number of shares outstanding

In [10]:
sql_script =\
'''
SELECT
    C.eventdate AS date,
    A.dsseccode AS security_code,
    A.dssecname AS security_name,
    A.primexchmnem AS primary_exchange,
    A.primqtinfocode AS refinitiv_code,
    B.isin AS isin_code,
    C.numshrs AS shares_outstanding
    
FROM
    tr_ds_equities.ds2security A

INNER JOIN
    tr_ds_equities.ds2isinchg B
ON
    A.dsseccode = B.dsseccode

INNER JOIN
    tr_ds_equities.ds2numshares C
ON
    A.primqtinfocode = C.infocode
    
WHERE
    B.isin = '{}'
AND
    EXTRACT(YEAR FROM C.eventdate) = 2023

ORDER BY C.eventdate
'''

In [11]:
# Extract shares outstanding
db.raw_sql(sql_script.format(ISIN))

,date,security_code,security_name,primary_exchange,refinitiv_code,isin_code,shares_outstanding
0,2023-01-09,29765.0,APPLE,NAS,72990.0,US0378331005,15836211.20
1,2023-01-20,29765.0,APPLE,NAS,72990.0,US0378331005,15821946.88
2,2023-03-31,29765.0,APPLE,NAS,72990.0,US0378331005,15723404.80
3,2023-04-21,29765.0,APPLE,NAS,72990.0,US0378331005,15728701.44
4,2023-07-01,29765.0,APPLE,NAS,72990.0,US0378331005,15647866.88
5,2023-07-21,29765.0,APPLE,NAS,72990.0,US0378331005,15634232.32


By Caden Lee 2023.